<a href="https://colab.research.google.com/github/MatthewRomanishin/courses/blob/main/PYDNN/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/keras-team/keras-tuner/
!pip install -U keras-tuner

In [ ]:
import numpy as np
import pandas as pd
from kerastuner import RandomSearch, Hyperband, BayesianOptimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline 
import os
""" посмотрим, какие файлы храняться в директории """
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/kaggle/CNN'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/gdrive/My Drive/kaggle/CNN/test.npy
/content/gdrive/My Drive/kaggle/CNN/sample_submission.csv
/content/gdrive/My Drive/kaggle/CNN/train.npy
/content/gdrive/My Drive/kaggle/CNN/train_label.npy


In [ ]:
# Названия классов из набора данных CIFAR-10
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [ ]:
"""Так как данные храняться не в формате таблиц, а в формате многомерных тензоров numpy,
то применим для загрузки данных функцию numpy load()"""
X_train = np.load('/content/gdrive/My Drive/kaggle/CNN/train.npy')
Y_train = np.load('/content/gdrive/My Drive/kaggle/CNN/train_label.npy')
X_test = np.load('/content/gdrive/My Drive/kaggle/CNN/test.npy')
X_train.shape, Y_train.shape, X_test.shape
x_train = X_train / 255
x_test = X_test / 255

In [ ]:
y_train = utils.to_categorical(Y_train, 10)

In [ ]:
def build_model(hp):
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
    activation_choice2 = hp.Choice('activation1', values=['relu','swish'])    

    model = Sequential()
    model.add(Conv2D(filters=hp.Int('filters',
                                    min_value=32,
                                    max_value=64,
                                    step=16),
              activation= activation_choice2,
              padding='valid',
              kernel_size=(3,3)
              ))
    
    model.add(Conv2D(filters=hp.Int('filters',
                                min_value=32,
                                max_value=64,
                                step=16),
            activation= activation_choice2,
            padding='valid',
            kernel_size=(3,3)
            ))
    
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Dropout(hp.Float('dropout',
                             min_value=0.1,
                             max_value=0.5,
                             step=0.2)))
    
    model.add(Conv2D(filters=hp.Int('filters',
                                min_value=32,
                                max_value=64,
                                step=16),
            activation= activation_choice2,
            padding='same',
            kernel_size= (3,3)
            ))

    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Dropout(hp.Float('dropout',
                             min_value=0.1,
                             max_value=0.5,
                             step=0.2)))
    
    model.add(Flatten())
    
    model.add(Dense(1024,activation= activation_choice2))

    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','nadam','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=1000,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

INFO:tensorflow:Reloading Oracle from existing project test_directory/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from test_directory/untitled_project/tuner0.json


In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Epoch 1/20
175/175 [==============================] - 4s 22ms/step - loss: 2.2931 - accuracy: 0.1106 - val_loss: 2.2791 - val_accuracy: 0.1258
Epoch 2/20
175/175 [==============================] - 4s 21ms/step - loss: 2.2413 - accuracy: 0.1562 - val_loss: 2.1800 - val_accuracy: 0.2362
Epoch 3/20
175/175 [==============================] - 4s 21ms/step - loss: 2.0846 - accuracy: 0.2460 - val_loss: 2.0282 - val_accuracy: 0.2641
Epoch 4/20
175/175 [==============================] - 4s 21ms/step - loss: 1.9985 - accuracy: 0.2732 - val_loss: 1.9697 - val_accuracy: 0.2862
Epoch 5/20
175/175 [==============================] - 4s 21ms/step - loss: 1.9537 - accuracy: 0.2934 - val_loss: 1.9136 - val_accuracy: 0.3152
Epoch 6/20
175/175 [==============================] - 4s 21ms/step - loss: 1.9177 - accuracy: 0.3127 - val_loss: 1.9027 - val_accuracy: 0.3260
Epoch 7/20
175/175 [==============================] - 4s 21ms/step - loss: 1.8749 - accuracy: 0.3315 - val_loss: 1.8235 - val_accuracy: 0.3576

Epoch 1/20
175/175 [==============================] - 5s 26ms/step - loss: 1.6380 - accuracy: 0.4055 - val_loss: 1.2956 - val_accuracy: 0.5448
Epoch 2/20
175/175 [==============================] - 4s 25ms/step - loss: 1.1928 - accuracy: 0.5752 - val_loss: 1.0462 - val_accuracy: 0.6302
Epoch 3/20
175/175 [==============================] - 4s 25ms/step - loss: 0.9848 - accuracy: 0.6526 - val_loss: 0.9013 - val_accuracy: 0.6881
Epoch 4/20
175/175 [==============================] - 4s 25ms/step - loss: 0.8672 - accuracy: 0.6966 - val_loss: 0.8424 - val_accuracy: 0.7070
Epoch 5/20
175/175 [==============================] - 4s 25ms/step - loss: 0.7627 - accuracy: 0.7309 - val_loss: 0.7730 - val_accuracy: 0.7362
Epoch 6/20
175/175 [==============================] - 4s 26ms/step - loss: 0.6808 - accuracy: 0.7603 - val_loss: 0.7170 - val_accuracy: 0.7561
Epoch 7/20
175/175 [==============================] - 4s 26ms/step - loss: 0.6002 - accuracy: 0.7898 - val_loss: 0.6998 - val_accuracy: 0.7588

Epoch 1/20
175/175 [==============================] - 4s 25ms/step - loss: 1.9694 - accuracy: 0.3642 - val_loss: 1.4001 - val_accuracy: 0.5044
Epoch 2/20
175/175 [==============================] - 4s 24ms/step - loss: 1.3188 - accuracy: 0.5421 - val_loss: 1.3122 - val_accuracy: 0.5475
Epoch 3/20
175/175 [==============================] - 4s 24ms/step - loss: 1.1964 - accuracy: 0.5860 - val_loss: 1.1355 - val_accuracy: 0.5953
Epoch 4/20
175/175 [==============================] - 4s 24ms/step - loss: 1.1196 - accuracy: 0.6117 - val_loss: 1.0867 - val_accuracy: 0.6243
Epoch 5/20
175/175 [==============================] - 4s 24ms/step - loss: 1.0601 - accuracy: 0.6312 - val_loss: 0.9680 - val_accuracy: 0.6683
Epoch 6/20
175/175 [==============================] - 4s 24ms/step - loss: 0.9946 - accuracy: 0.6534 - val_loss: 0.9471 - val_accuracy: 0.6727
Epoch 7/20
175/175 [==============================] - 4s 24ms/step - loss: 0.9184 - accuracy: 0.6794 - val_loss: 0.9500 - val_accuracy: 0.6758